<a href="https://colab.research.google.com/github/ToshikiNakamura0412/my_ipynb/blob/master/feature_matching_with_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install opencv-python
!pip install ultralytics

In [24]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

def extract_features_yolo(detector, img_path):
  # YOLO
  model = YOLO("yolov8n.pt")
  results = model.predict(source=img_path, save=True)

  # mask
  img = cv2.imread(img_path)
  mask = np.full(img.shape[:2], 255, dtype=np.uint8)
  for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    mask[y1:y2, x1:x2] = 0

  # extraction
  keypoints, descriptors = detector.detectAndCompute(img, mask)
  img_with_keypoints = cv2.drawKeypoints(img, keypoints, None, color=(0, 255, 0), flags=0)

  # viz
  cv2_imshow(img_with_keypoints)

def calc_descriptors_yolo(detector, img_path):
  # YOLO
  model = YOLO("yolov8n.pt")
  results = model.predict(source=img_path, save=True)

  # mask
  img = cv2.imread(img_path)
  mask = np.full(img.shape[:2], 255, dtype=np.uint8)
  for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    mask[y1:y2, x1:x2] = 0

  # extraction
  keypoints, descriptors = detector.detectAndCompute(img, mask)

  return descriptors

def calc_descriptors(detector, img_path):
  img = cv2.imread(img_path)
  keypoints, descriptors = detector.detectAndCompute(img, None)

  return descriptors

def calc_similarity(descriptor1, descriptor2):
  matcher = cv2.BFMatcher()
  matches = matcher.knnMatch(descriptor1, descriptor2, k=2)
  dist = [m.distance for m, n in matches if m.distance < 0.75 * n.distance]
  return np.mean(dist)

In [25]:
detector = cv2.ORB_create()

# YOLO
source_descriptor = calc_descriptors_yolo(detector, "image_0.png")
target_descriptor = calc_descriptors_yolo(detector, "image_1.png")
similarity = calc_similarity(source_descriptor, target_descriptor)
print(similarity)

# No YOLO
source_descriptor = calc_descriptors(detector, "image_0.png")
target_descriptor = calc_descriptors(detector, "image_1.png")
similarity = calc_similarity(source_descriptor, target_descriptor)
print(similarity)


image 1/1 /content/image_0.png: 384x640 10 persons, 1 bus, 148.3ms
Speed: 2.6ms preprocess, 148.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict15

image 1/1 /content/image_1.png: 384x640 9 persons, 1 bus, 141.6ms
Speed: 2.8ms preprocess, 141.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict16
184.92252750128088
188.49818210278528


In [26]:
detector = cv2.AKAZE_create()

# YOLO
source_descriptor = calc_descriptors_yolo(detector, "image_0.png")
target_descriptor = calc_descriptors_yolo(detector, "image_1.png")
similarity = calc_similarity(source_descriptor, target_descriptor)
print(similarity)

# No YOLO
source_descriptor = calc_descriptors(detector, "image_0.png")
target_descriptor = calc_descriptors(detector, "image_1.png")
similarity = calc_similarity(source_descriptor, target_descriptor)
print(similarity)


image 1/1 /content/image_0.png: 384x640 10 persons, 1 bus, 153.1ms
Speed: 2.3ms preprocess, 153.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict17

image 1/1 /content/image_1.png: 384x640 9 persons, 1 bus, 141.4ms
Speed: 2.5ms preprocess, 141.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict18
266.31062785515417
266.0044492056476
